In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from scripts import utils, schemas

from urllib.request import urlopen
from PIL import Image # type: ignore
from mplsoccer import PyPizza, add_image, FontManager # type: ignore

pd.set_option('display.max_columns', None)

In [3]:
path = "https://raw.githubusercontent.com/griffisben/Wyscout_Prospect_Research/main/Main%20App/Brasileir%C3%A3o%202024.csv"

In [4]:
data = pd.read_csv(path)

In [5]:
data

,Player,Full name,Wyscout id,Team,Team within selected timeframe,Team logo,Competition,Position,Primary position,"Primary position, %",Secondary position,"Secondary position, %",Third position,"Third position, %",Age,Birthday,Market value,Contract expires,Matches played,Minutes played,Goals,xG,Assists,xA,Duels per 90,"Duels won, %",Birth country,Passport country,Foot,Height,Weight,On loan,Successful defensive actions per 90,Defensive duels per 90,"Defensive duels won, %",Aerial duels per 90,"Aerial duels won, %",Sliding tackles per 90,PAdj Sliding tackles,Shots blocked per 90,Interceptions per 90,PAdj Interceptions,Fouls per 90,Yellow cards,Yellow cards per 90,Red cards,Red cards per 90,Successful attacking actions per 90,Goals per 90,Non-penalty goals,Non-penalty goals per 90,xG per 90,Head goals,Head goals per 90,Shots,Shots per 90,"Shots on target, %","Goal conversion, %",Assists per 90,Crosses per 90,"Accurate crosses, %",Crosses from left flank per 90,"Accurate crosses from left flank, %",Crosses from right flank per 90,"Accurate crosses from right flank, %",Crosses to goalie box per 90,Dribbles per 90,"Successful dribbles, %",Offensive duels per 90,"Offensive duels won, %",Touches in box per 90,Progressive runs per 90,Accelerations per 90,Received passes per 90,Received long passes per 90,Fouls suffered per 90,Passes per 90,"Accurate passes, %",Forward passes per 90,"Accurate forward passes, %",Back passes per 90,"Accurate back passes, %",Short / medium passes per 90,"Accurate short / medium passes, %",Long passes per 90,"Accurate long passes, %","Average pass length, m","Average long pass length, m",xA per 90,Shot assists per 90,Second assists per 90,Third assists per 90,Smart passes per 90,"Accurate smart passes, %",Key passes per 90,Passes to final third per 90,"Accurate passes to final third, %",Passes to penalty area per 90,"Accurate passes to penalty area, %",Through passes per 90,"Accurate through passes, %",Deep completions per 90,Deep completed crosses per 90,Progressive passes per 90,"Accurate progressive passes, %","Accurate vertical passes, %",Vertical passes per 90,Conceded goals,Conceded goals per 90,Shots against,Shots against per 90,Clean sheets,"Save rate, %",xG against,xG against per 90,Prevented goals,Prevented goals per 90,Back passes received as GK per 90,Exits per 90,Aerial duels per 90.1,Free kicks per 90,Direct free kicks per 90,"Direct free kicks on target, %",Corners per 90,Penalties taken,"Penalty conversion, %"
0,Fábio,Fábio Deivson Lopes Maciel,8961,Fluminense,Fluminense,https://cdn5.wyscout.com/photos/team/public/24...,Serie A,GK,GK,100,NaN,0,NaN,0,43,1980-09-30,0,2025-12-31,7,731,0,0.00,0,0.00,0.37,100.00,Brazil,"['Brazil', 'Italy']",right,190,93,False,1.85,0.00,0.00,0.25,100.00,0.00,0.00,0.00,1.85,3.17,0.00,0,0.00,0,0.0,0.00,0.0,0,0.0,0.00,0,0.0,0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.83,0.00,0.00,16.74,87.50,7.51,81.97,0.12,0.00,12.80,93.27,3.94,68.75,22.98,35.93,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,3.45,57.14,93.65,7.76,13,1.60,43,5.29,0,69.77,12.51,1.54,-0.49,-0.060,0.0,1.11,0.25,NaN,NaN,NaN,0.00,0,0
1,J. Arias,Jhon Adolfo Arias Andrade,449822,Fluminense,Fluminense,https://cdn5.wyscout.com/photos/team/public/24...,Serie A,"LAMF, RW, RAMF",LAMF,51,RW,18,RAMF,17,26,1997-09-21,0,2026-06-30,7,731,0,1.53,1,0.59,22.04,51.96,Colombia,['Colombia'],right,168,70,False,6.89,6.40,69.23,1.60,38.46,0.00,0.00,0.12,2.46,4.23,1.11,1,0.12,0,0.0,4.06,0.0,0,0.0,0.19,0,0.0,14,1.72,35.71,0.0,0.12,1.72,21.43,1.11,22.22,0.62,20.00,0.12,5.29,55.81,11.94,47.42,2.34,2.95,1.35,29.18,0.98,2.22,35.46,88.19,6.16,80.00,9.97,96.30,33.61,91.94,0.25,0.00,13.71,12.63,0.07,0.74,0.12,0.0,0.25,50.0,0.49,2.34,89.47,1.48,25.00,0.25,50.0,0.49,0.25,1.85,80.00,83.75,9.85,0,NaN,0,NaN,0,0.00,0.00,0.00,NaN,NaN,0.0,NaN,1.60,0.25,0.00,0.0,0.62,0,0
2,Marquinhos,Marcus Vinicius Oliveira Alencar,-118586,Fluminense,Fluminense,https://cdn5.wyscout.com/

In [ ]:
fotmob = soccerdata.FotMob(leagues='INT-European Championship', seasons=2024)

In [ ]:
schedule = fotmob.read_schedule().reset_index()

In [ ]:
topstats = fotmob.read_team_match_stats("Top stats").reset_index()

In [21]:
events = fotmob.read_match_details(schedule)

In [37]:
def process_events(data):
    # Initialize an empty list to collect all match data and match stats
    all_match_data = []
    all_match_stats = []

    for i in data:
        general = i['general']
    # Extract the required match data
        match_data = {
            'matchId': general.get('matchId'),
            'matchName': general.get('matchName'),
            'leagueRoundName': general.get('leagueRoundName'),
            'countryCode': general.get('countryCode'),
            'home_team_name': general['homeTeam'].get('name'),
            'away_team_name': general['awayTeam'].get('name'),
            'home_team_id': general['homeTeam'].get('id'),
            'away_team_id': general['awayTeam'].get('id')
        }
        
        # Append the match data to the list
        all_match_data.append(match_data)

        # Extract the match stats
        if 'shotmap' in i['content']:
            shots = i['content']['shotmap']['shots']
            for shot in shots:
                shot['matchId'] = general.get('matchId')
                shot['teamName'] = general['homeTeam']['name'] if shot['teamId'] == general['homeTeam']['id'] else general['awayTeam']['name']
                all_match_stats.append(shot)
    
    # Convert the lists of match data and match stats to DataFrames
    match_df = pd.DataFrame(all_match_data)
    stats_df = pd.json_normalize(all_match_stats)
    
    return match_df, stats_df

In [38]:
match_df, stats_df = process_events(events)

In [39]:
stats_df

,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,isOnTarget,blockedX,blockedY,goalCrossedY,goalCrossedZ,expectedGoals,expectedGoalsOnTarget,shotType,situation,period,isOwnGoal,isSavedOffLine,isFromInsideBox,firstName,lastName,fullName,teamColor,matchId,teamName,onGoalShot.x,onGoalShot.y,onGoalShot.zoomRatio
0,2696028051,Goal,8570,1152455,Florian Wirtz,87.748529,32.780000,10,NaN,False,True,NaN,NaN,36.973750,0.282526,0.102611,0.3744,RightFoot,RegularPlay,FirstHalf,False,False,False,Florian,Wirtz,Florian Wirtz,#464646,4043837,Germany,0.213294,0.074742,1.000000
1,2696028913,Goal,8570,1156141,Jamal Musiala,93.000000,36.287500,19,NaN,False,True,NaN,NaN,35.982500,2.074000,0.175834,0.3379,RightFoot,RegularPlay,FirstHalf,False,False,True,Jamal,Musiala,Jamal Musiala,#464646,4043837,Germany,0.475529,0.548677,1.000000
2,2696029565,AttemptSaved,8570,749736,Kai Havertz,86.997059,32.398750,28,NaN,False,True,103.672414,33.695000,36.821250,0.488000,0.138585,0.3661,LeftFoot,FreeKick,FirstHalf,False,False,False,Kai,Havertz,Kai Havertz,#464646,4043837,Germany,0.253638,0.129101,1.000000
3,2696030523,AttemptSaved,8570,178818,Ilkay Gündogan,97.763158,31.560000,42,NaN,False,True,103.672414,33.695000,35.601250,0.083474,0.099804,0.1761,Header,RegularPlay,FirstHalf,False,False,True,Ilkay,Gündogan,Ilkay Gündogan,#464646,4043837,Germany,0.576389,0.022083,1.000000
4,2696030659,Goal,8570,749736,Kai Havertz,94.000000,34.000000,45,1.0,False,True,NaN,NaN,35.677500,0.892526,0.788400,0.9606,LeftFoot,Penalty,FirstHalf,False,False,True,Kai,Havertz,Kai Havertz,#464646,4043837,Germany,0.556217,0.236118,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2699291251,AttemptSaved,8570,276791,Niclas Füllkrug,95.929825,31.255000,117,NaN,False,True,103.862069,34.000000,36.211250,0.976000,0.058766,0.1843,Header,RegularPlay,SecondHalfExtra,False,False,True,Niclas,Füllkrug,Niclas Füllkrug,#464646,4043979,Germany,0.415013,0.258201,1.000000
1226,2699291419,Goal,6720,574645,Mikel Merino,97.666667,34.686250,119,NaN,False,True,NaN,NaN,37.126250,1.181474,0.302742,0.8161,Header,RegularPlay,SecondHalfExtra,False,False,True,Mikel,Merino,Mikel Merino,#c62035,4043979,Spain,0.172950,0.312559,1.000000
1227,2699291569,AttemptSaved,8570,460632,Joshua Kimmich,85.494118,29.043333,120,2.0,True,True,94.096491,30.721250,33.618750,1.220000,0.070082,NaN,RightFoot,RegularPlay,SecondHalfExtra,False,False,False,Joshua,Kimmich,Joshua Kimmich,#464646,4043979,Germany,1.100860,0.322751,1.000000
1228,2699291603,Miss,6720,881771,Ferran Torres,96.894737,23.859108,120,2.0,False,False,102.913793,40.301429,45.612229,0.622842,0.400407,NaN,RightFoot,FastBreak,SecondHalfExtra,False,False,True,Ferran,Torres,Ferran Torres,#c62035,4043979,Spain,0.000000,0.053637,0.325519


In [26]:
pd.json_normalize(events[0]['content']['shotmap']['shots'])

,id,eventType,teamId,playerId,playerName,x,y,min,minAdded,isBlocked,isOnTarget,blockedX,blockedY,goalCrossedY,goalCrossedZ,expectedGoals,expectedGoalsOnTarget,shotType,situation,period,isOwnGoal,isSavedOffLine,isFromInsideBox,firstName,lastName,fullName,teamColor,onGoalShot.x,onGoalShot.y,onGoalShot.zoomRatio
0,2696028051,Goal,8570,1152455,Florian Wirtz,87.748529,32.780000,10,NaN,False,True,NaN,NaN,36.973750,0.282526,0.102611,0.3744,RightFoot,RegularPlay,FirstHalf,False,False,False,Florian,Wirtz,Florian Wirtz,#464646,0.213294,0.074742,1.000000
1,2696028913,Goal,8570,1156141,Jamal Musiala,93.000000,36.287500,19,NaN,False,True,NaN,NaN,35.982500,2.074000,0.175834,0.3379,RightFoot,RegularPlay,FirstHalf,False,False,True,Jamal,Musiala,Jamal Musiala,#464646,0.475529,0.548677,1.000000
2,2696029565,AttemptSaved,8570,749736,Kai Havertz,86.997059,32.398750,28,NaN,False,True,103.672414,33.695000,36.821250,0.488000,0.138585,0.3661,LeftFoot,FreeKick,FirstHalf,False,False,False,Kai,Havertz,Kai Havertz,#464646,0.253638,0.129101,1.000000
3,2696030523,AttemptSaved,8570,178818,Ilkay Gündogan,97.763158,31.560000,42,NaN,False,True,103.672414,33.695000,35.601250,0.083474,0.099804,0.1761,Header,RegularPlay,FirstHalf,False,False,True,Ilkay,Gündogan,Ilkay Gündogan,#464646,0.576389,0.022083,1.000000
4,2696030659,Goal,8570,749736,Kai Havertz,94.000000,34.000000,45,1.0,False,True,NaN,NaN,35.677500,0.892526,0.788400,0.9606,LeftFoot,Penalty,FirstHalf,False,False,True,Kai,Havertz,Kai Havertz,#464646,0.556217,0.236118,1.000000
5,2696031019,AttemptSaved,8570,276738,Antonio Rüdiger,73.024757,31.255000,51,NaN,False,True,102.818966,33.695000,36.973750,0.693474,0.026680,0.2263,RightFoot,RegularPlay,SecondHalf,False,False,False,Antonio,Rüdiger,Antonio Rüdiger,#464646,0.213294,0.183459,1.000000
6,2696031053,AttemptSaved,8570,276738,Antonio Rüdiger,93.400000,31.255000,52,NaN,True,True,96.605263,32.475000,35.753750,1.220000,0.017594,NaN,Header,FromCorner,SecondHalf,False,False,True,Antonio,Rüdiger,Antonio Rüdiger,#464646,0.536045,0.322751,1.000000
7,2696031251,AttemptSaved,8570,167008,Pascal Gross,84.512136,46.312866,57,NaN,True,True,87.963235,44.140892,32.475000,1.220000,0.033208,NaN,RightFoot,FromCorner,SecondHalf,False,False,False,Pascal,Gross,Pascal Gross,#464646,1.403439,0.322751,1.000000
8,2696031263,AttemptSaved,8570,460632,Joshua Kimmich,83.498544,23.018344,57,NaN,True,True,93.000000,27.026190,35.525000,1.220000,0.033395,NaN,RightFoot,FromCorner,SecondHalf,False,False,False,Joshua,Kimmich,Joshua Kimmich,#464646,0.596561,0.322751,1.000000
9,2696031295,Miss,8570,1152455,Florian Wirtz,97.763158,27.614524,58,NaN,False,False,NaN,NaN,31.788750,4.584690,0.097917,NaN,RightFoot,RegularPlay,SecondHalf,False,False,True,Florian,Wirtz,Florian Wirtz,#464646,1.326645,0.677249,0.558380


In [ ]:
schedule.keys()

In [22]:
pd.json_normalize(events[0]['content']['lineup']['lineup'][0])

,teamId,teamName,bench,coach,players,lineup
0,8570,Germany,"[{'id': '167008', 'positionId': 0, 'position':...","[{'id': '36567', 'name': {'firstName': 'Julian...","[[{'id': '27299', 'positionId': 11, 'position'...",4 - 2 - 3 - 1


In [ ]:
pd.json_normalize(events['content'][0][])

In [ ]:
pd.DataFrame(events['content'][0])

In [ ]:
league_info_url = 'https://raw.githubusercontent.com/griffisben/Wyscout_Prospect_Research/main/league_info_lookup.csv'
leagues = utils.read_csv(league_info_url)
selected_league = "Premier League"
selected_season = "23-24"
full_league_name = f"{selected_league} {selected_season}"

In [ ]:
league_season_data = utils.read_csv2((f'https://raw.githubusercontent.com/griffisben/Wyscout_Prospect_Research/main/Main%20App/{full_league_name.replace(" ","%20").replace("ü","u").replace("ó","o").replace("ö","o")}.csv'))
league_season_data = league_season_data[list(schemas.column_mapping().values())]
league_season_data['Lig'] = f'{selected_league}'

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)

# Compute pie slices
N = 10
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
radii = 10 * np.random.rand(N)
width = np.pi / 4 * np.random.rand(N)
colors = plt.cm.viridis(radii / 10.)

ax = plt.subplot(projection='polar')
ax.bar(theta, radii, width=width, bottom=0.0, color=colors, alpha=0.5)

plt.show()